<a href="https://colab.research.google.com/github/gitclub-data/Agentic-AI/blob/master/Natual_language_Query_Answers_From_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing all the nessary libraries

In [ ]:
!pip install typing_extensions
!pip install langgraph
!pip install langchain-community
!pip install -U "langchain[google-genai]"

Connect With Google Drive

In [ ]:
#connect colab with google drive
from google.colab import drive
drive.mount('/content/drive')

Create Real and Dummy Dataset Real for store real data to query, dummy store very much less but good quality data for validation purposes

In [ ]:
# create a small database to store some data
import sqlite3
conn = sqlite3.connect('real_database.db')
dummy_conn = sqlite3.connect('dummy_database.db')

In [ ]:
cursor = conn.cursor()
dummy_cursor = dummy_conn.cursor()

In [ ]:
create_users_table = '''
    CREATE TABLE IF NOT EXISTS users (
        user_id INTEGER PRIMARY KEY,
        user_gender BOOLEAN,
        bucketized_user_age INTEGER,
        user_occupation_label INTEGER,
        user_occupation_text TEXT,
        user_zip_code INTEGER
    )
'''
cursor.execute(create_users_table)
dummy_cursor.execute(create_users_table)

In [ ]:
create_ratings_table = '''
    CREATE TABLE IF NOT EXISTS ratings (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        movie_id INTEGER,
        user_id INTEGER,
        user_rating INTEGER,
        timestamp INTEGER
    )
'''
cursor.execute(create_ratings_table)
dummy_cursor.execute(create_ratings_table)

In [ ]:
create_movies_table = '''
    CREATE TABLE IF NOT EXISTS movies (
        movie_id INTEGER PRIMARY KEY,
        movie_title TEXT,
        poster_url TEXT
    )
'''
cursor.execute(create_movies_table)
dummy_cursor.execute(create_movies_table)

MovieLens 1M with Posters & Metadata (Kaggle Dataset)

In [ ]:
import pandas as pd

# Adjust paths if your files are in a subfolder
ratings_df = pd.read_csv('/content/drive/MyDrive/Movies_Data/ratings.csv')
movies_df = pd.read_csv('/content/drive/MyDrive/Movies_Data/movies.csv')
users_df = pd.read_csv('/content/drive/MyDrive/Movies_Data/users.csv')

ENTERED FULL DATA IN DUMMY DATABASE FOR NOW ONLY FOR POC PURPOSE

In [ ]:
ratings_df.to_sql('ratings', conn, if_exists='append', index=False)
movies_df.to_sql('movies', conn, if_exists='append', index=False)
users_df.to_sql('users', conn, if_exists='append', index=False)

ratings_df.to_sql('ratings', dummy_conn, if_exists='append', index=False)
movies_df.to_sql('movies', dummy_conn, if_exists='append', index=False)
users_df.to_sql('users', dummy_conn, if_exists='append', index=False)

dummy_conn.commit()
dummy_conn.close()

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('real_database.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM users')
rows = cursor.fetchall()

# Print each row
for row in rows:
    print(row)
    break
conn.commit()
conn.close()

In [ ]:
#initial table description

CREATED THIS JSON OF TABLE INFORMATION TO GET THE TABLE SCHEMA IN STRUCTURED FORMAT -> NEEDED AS INPUT SO THAT MODEL CAN EXCECTLY KNOW WHAT TYPE OF QUERY IT HAS TO GENERATE(Used Previously when not used embeddings..)

In [ ]:
table_info = {
    "name": "tables_information",
    "tables": [
        {
            "table_name": "users",
            "table_description": "This table contains records of users who have watched the movie in the past.",
            "columns": [
                {
                    "column_name": "user_id",
                    "column_type": "INTEGER",
                    "column_description": "Unique identifier for each user"
                },
                {
                    "column_name": "user_gender",
                    "column_type": "BOOLEAN",
                    "column_description": "True = Male, False = Female"
                },
                {
                    "column_name": "bucketized_user_age",
                    "column_type": "INTEGER",
                    "column_description": (
                        "Age group bucket:\n"
                        "- 1: Under 18\n"
                        "- 18: 18–24\n"
                        "- 25: 25–34\n"
                        "- 35: 35–44\n"
                        "- 45: 45–49\n"
                        "- 50: 50–55\n"
                        "- 56: 56-.."
                    )
                },
                {
                    "column_name": "user_occupation_label",
                    "column_type": "INTEGER",
                    "column_description": (
                        "Integer code representing occupation:\n"
                        "- 0: other or not specified\n"
                        "- 1: academic/educator\n"
                        "- 2: artist\n"
                        "- 3: clerical/admin\n"
                        "- 4: college/grad student\n"
                        "- 5: customer service\n"
                        "- 6: doctor/health care\n"
                        "- 7: executive/managerial\n"
                        "- 8: farmer\n"
                        "- 9: homemaker\n"
                        "- 10: K-12 student\n"
                        "- 11: lawyer\n"
                        "- 12: programmer\n"
                        "- 13: retired\n"
                        "- 14: sales/marketing\n"
                        "- 15: scientist\n"
                        "- 16: self-employed\n"
                        "- 17: technician/engineer\n"
                        "- 18: tradesman/craftsman\n"
                        "- 19: unemployed\n"
                        "- 20: writer"
                    )
                },
                {
                    "column_name": "user_occupation_text",
                    "column_type": "TEXT",
                    "column_description": "Text description of the occupation (e.g. Writer)"
                },
                {
                    "column_name": "user_zip_code",
                    "column_type": "INTEGER",
                    "column_description": "ZIP code provided voluntarily by the user"
                }
            ]
        },
        {
            "table_name": "ratings",
            "table_description": "This table captures the ratings provided by users for specific movies.",
            "columns": [
                {
                    "column_name": "user_id",
                    "column_type": "INTEGER",
                    "column_description": "Foreign key referring to users table."
                },
                {
                    "column_name": "movie_id",
                    "column_type": "INTEGER",
                    "column_description": "Foreign key referring to movies table."
                },
                {
                    "column_name": "user_rating",
                    "column_type": "INTEGER",
                    "column_description": "Rating from 1 to 5 (whole stars only)."
                },
                {
                    "column_name": "timestamp",
                    "column_type": "INTEGER",
                    "column_description": "Unix timestamp when the rating was submitted."
                }
            ]
        },
        {
            "table_name": "movies",
            "table_description": "This table includes metadata related to the movies.",
            "columns": [
                {
                    "column_name": "movie_id",
                    "column_type": "INTEGER",
                    "column_description": "Unique movie identifier"
                },
                {
                    "column_name": "movie_title",
                    "column_type": "TEXT",
                    "column_description": "Title of the movie (with release year)"
                },
                {
                    "column_name": "poster_url",
                    "column_type": "TEXT",
                    "column_description": "Direct URL to the movie's poster image"
                }
            ]
        }
    ]
}

In [ ]:
#plan of action: For getting table information from embeddings..

In [ ]:
# use faiss vector store
# ->> type 1 -> table embedding stores whole table record (table and its column information in vector store).
#  (used for what table we will be needing for actually getting the data)
# ->> type 2 -> used to store the table name its column name and description of the column





# ->> getting the result by k nearest neighbour algo.. but it is impossible to determine if we got all the necessary details
# in the first try or may be we can find that out. so we let the model check if not find all the nessary details we are going
#  to search the table again untill reached the max limit and break out and sent can't find neccessary columns message to the output.

In [ ]:
!pip install -qU langchain-community faiss-cpu sentence-transformers

Used a Open Source Model Because I do not have that much money to get a paid subscription

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# Define embedding function that LangChain expects
embedding_function = lambda texts: model.encode(texts, convert_to_numpy=True)

# Create FAISS index
sample_embedding = model.encode("hello world")
embedding_dimension = len(sample_embedding)
index = faiss.IndexFlatL2(embedding_dimension)

# Create FAISS vector store
vector_store = FAISS(
    embedding_function=embedding_function,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

Created Documents and saved them into vector store (column as well as tables)

In [ ]:
#table schema
from langchain_core.documents import Document

user_schema = Document(
    page_content="""
Table Name: users
Description: This table contains records of users who have watched the movie in the past.

Columns:
- user_id (INTEGER): Unique identifier for each user
- user_gender (BOOLEAN): True = Male, False = Female
- bucketized_user_age (INTEGER): Age group bucket:
  - 1: Under 18
  - 18: 18–24
  - 25: 25–34
  - 35: 35–44
  - 45: 45–49
  - 50: 50–55
  - 56: 56 and above
- user_occupation_label (INTEGER): Integer code for occupation
  - 0: other or not specified
  - 1–20: specific occupations like artist, lawyer, student, etc.
- user_occupation_text (TEXT): Occupation description (e.g. Writer)
- user_zip_code (INTEGER): ZIP code provided voluntarily by the user
""",
    metadata={"source": "table_schema", "table_name": "users"}
)

ratings_schema = Document(
    page_content="""
Table Name: ratings
Description: This table captures the ratings provided by users for specific movies.

Columns:
- user_id (INTEGER): Foreign key referring to users table
- movie_id (INTEGER): Foreign key referring to movies table
- user_rating (INTEGER): Rating from 1 to 5 (whole stars only)
- timestamp (INTEGER): Unix timestamp when the rating was submitted
""",
    metadata={"source": "table_schema", "table_name": "ratings"}
)

movies_schema = Document(
    page_content="""
Table Name: movies
Description: This table includes metadata related to the movies.

Columns:
- movie_id (INTEGER): Unique movie identifier
- movie_title (TEXT): Title of the movie (with release year)
- poster_url (TEXT): Direct URL to the movie's poster image
""",
    metadata={"source": "table_schema", "table_name": "movies"}
)

In [ ]:
user_id_doc = Document(
    page_content="- user_id (INTEGER): Unique identifier for each user",
    metadata={
        "source": "column_schema",
        "table_name": "users",
        "table_description": "This table contains records of users who have watched the movie in the past.",
        "column_name": "user_id"
    }
)

user_gender_doc = Document(
    page_content="- user_gender (BOOLEAN): True = Male, False = Female",
    metadata={
        "source": "column_schema",
        "table_name": "users",
        "table_description": "This table contains records of users who have watched the movie in the past.",
        "column_name": "user_gender"
    }
)

bucketized_user_age_doc = Document(
    page_content="""- bucketized_user_age (INTEGER): Age group bucket:
  - 1: Under 18
  - 18: 18–24
  - 25: 25–34
  - 35: 35–44
  - 45: 45–49
  - 50: 50–55
  - 56: 56 and above""",
    metadata={
        "source": "column_schema",
        "table_name": "users",
        "table_description": "This table contains records of users who have watched the movie in the past.",
        "column_name": "bucketized_user_age"
    }
)

user_occupation_label_doc = Document(
    page_content="""- user_occupation_label (INTEGER): Integer code for occupation
  - 0: other or not specified
  - 1–20: specific occupations like artist, lawyer, student, etc.""",
    metadata={
        "source": "column_schema",
        "table_name": "users",
        "table_description": "This table contains records of users who have watched the movie in the past.",
        "column_name": "user_occupation_label"
    }
)

user_occupation_text_doc = Document(
    page_content='- user_occupation_text (TEXT): Occupation description (e.g. Writer)',
    metadata={
        "source": "column_schema",
        "table_name": "users",
        "table_description": "This table contains records of users who have watched the movie in the past.",
        "column_name": "user_occupation_text"
    }
)

user_zip_code_doc = Document(
    page_content='- user_zip_code (INTEGER): ZIP code provided voluntarily by the user',
    metadata={
        "source": "column_schema",
        "table_name": "users",
        "table_description": "This table contains records of users who have watched the movie in the past.",
        "column_name": "user_zip_code"
    }
)

In [ ]:
user_id__doc = Document(
    page_content="- user_id (INTEGER): Foreign key referring to users table",
    metadata={
        "source": "column_schema",
        "table_name": "ratings",
        "table_description": "This table captures the ratings provided by users for specific movies.",
        "column_name": "user_id"
    }
)

movie_id_doc = Document(
    page_content="- movie_id (INTEGER): Foreign key referring to movies table",
    metadata={
        "source": "column_schema",
        "table_name": "ratings",
        "table_description": "This table captures the ratings provided by users for specific movies.",
        "column_name": "movie_id"
    }
)

user_rating_doc = Document(
    page_content="- user_rating (INTEGER): Rating from 1 to 5 (whole stars only)",
    metadata={
        "source": "column_schema",
        "table_name": "ratings",
        "table_description": "This table captures the ratings provided by users for specific movies.",
        "column_name": "user_rating"
    }
)

timestamp_doc = Document(
    page_content="- timestamp (INTEGER): Unix timestamp when the rating was submitted",
    metadata={
        "source": "column_schema",
        "table_name": "ratings",
        "table_description": "This table captures the ratings provided by users for specific movies.",
        "column_name": "timestamp"
    }
)

In [ ]:
movie_id__doc = Document(
    page_content="- movie_id (INTEGER): Unique movie identifier",
    metadata={
        "source": "column_schema",
        "table_name": "movies",
        "table_description": "This table includes metadata related to the movies.",
        "column_name": "movie_id"
    }
)

movie_title_doc = Document(
    page_content="- movie_title (TEXT): Title of the movie (with release year)",
    metadata={
        "source": "column_schema",
        "table_name": "movies",
        "table_description": "This table includes metadata related to the movies.",
        "column_name": "movie_title"
    }
)

poster_url_doc = Document(
    page_content="- poster_url (TEXT): Direct URL to the movie's poster image",
    metadata={
        "source": "column_schema",
        "table_name": "movies",
        "table_description": "This table includes metadata related to the movies.",
        "column_name": "poster_url"
    }
)

In [ ]:
from langchain_core.documents import Document

MAX_TABLE_SCHEMA = 3
MAX_COLUMN_SCHEMA = 13

# Create Document objects (if not already)
raw_documents = [
    user_schema,
    ratings_schema,
    movies_schema,
    user_id__doc,
    user_gender_doc,
    bucketized_user_age_doc,
    user_occupation_label_doc,
    user_occupation_text_doc,
    user_zip_code_doc,
    user_id_doc,
    movie_id_doc,
    user_rating_doc,
    timestamp_doc,
    movie_id__doc,
    movie_title_doc,
    poster_url_doc
]

# Add documents to the FAISS vector store
vector_store.add_documents(raw_documents)

In [ ]:
vector_store.save_local("table_information")

Used a free tier model and you know what I needed my keys safe so i have stored in secrets 'key icon ON YOUR LEFT (for marvel-fans only)' (In colab)

In [ ]:
import os
from langchain.chat_models import init_chat_model
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
llm = init_chat_model("google_genai:gemini-2.0-flash")

Started Creating Our First Workflow (Related Table generation)

In [ ]:
from typing_extensions import Annotated
from typing_extensions import TypedDict
from langgraph.graph import START,END, StateGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

class TableState(TypedDict):
    question: str # user query
    relevant_tables : int  # no of relevant tables
    table_documents : list # tables documents which are going to fetch
    validation_result : dict # validation result
    resultant_tables: list # list of tables
    error_result : str # if the relevant tables can not found.
    vectorstore : FAISS

def get_relevant_tables(state : TableState):
  vector_store = state["vectorstore"]
  results = vector_store.similarity_search(
    state["question"],
    k=state["relevant_tables"],
    filter={"source": "table_schema"},
  )
  return {"table_documents":results}

In [ ]:
class QueryJsonOutput(TypedDict):
    """Generated SQL query."""
    enough_information: Annotated[bool, ..., "whether the available schema provides sufficient information to write a valid SQL query that could answer the question or not."]

In [ ]:
def validate_relevant_tables(state : TableState):
  system_prompt = """
You are a helpful assistant that evaluates whether a given set of database tables contains enough schema information to generate a valid SQL query that answers a natural language question **in full**.

You will be provided with:
- A natural language question.
- One or more database tables. Each table includes:
  - Table name and description
  - Column names, data types, and column descriptions

Assume that:
- All columns are populated with valid and correctly typed data.
- You are not required to write the SQL query or answer the question — only to determine whether it is possible to generate a complete query using the schema provided.

Your task is to:
1. Carefully analyze the natural language question to identify **all distinct information requirements** (e.g., filters, metrics, grouping, sorting).
2. Review the table schemas and ensure that **every required data point** is represented by one or more columns in the schema.
3. Consider relationships between tables only if they are explicitly defined or logically inferable from naming or descriptions.

Only if **every part** of the question can be addressed with the schema provided, return:
{{"enough_information": true}}

If **any required element is missing** (e.g., age of users, movie ratings, like counts, timestamps, etc.), return:
{{"enough_information": false}}

Constraints:
- You must evaluate the entire question in detail — partial answerability is not sufficient.
- Do not assume access to external data, hidden metadata, or business logic beyond the schema.
- Do not write or return SQL queries.
- Your output must be only a single JSON object.

Input format:
Question: {natural_language_question}

Table(s):
{table_name_and_description}

Example output:
{{"enough_information": true}}
"""

  table_information = ""
  for documents in state["table_documents"]:
    table_information += "\n"+documents.page_content

  print(table_information)

  prompt = ChatPromptTemplate.from_template(system_prompt)

  structured_llm = llm.with_structured_output(QueryJsonOutput)

  validation_chain = prompt | structured_llm

  result = validation_chain.invoke({"natural_language_question" : state["question"], "table_name_and_description": table_information})

  print(result)
  return {"validation_result" : result}

In [ ]:
def send_tables(state:TableState):
  tables = set()
  for document in state["table_documents"]:
    tables.add(document.metadata["table_name"])
  return {"resultant_tables" : list(tables)}

def not_possible_answer(state: TableState):
  return {"error_result":"result can not be found because It looks like you do not have enough data"}

In [ ]:
def transition_by_validation(state: TableState):
  if "enough_information" not in state["validation_result"]:
    raise Exception("llm is not returning correct json")
  if state["validation_result"]["enough_information"] is True:
    return "answer_found"
  else:
    if state["relevant_tables"]<MAX_TABLE_SCHEMA:
      return "increase_relevence and reiletrate"
    else:
      return "answer_can_not_found"

In [ ]:
def increase_relevance(state: TableState):
  result = state["relevant_tables"] + 2
  return {"relevant_tables" : result}

In [ ]:
# step 1 -> send the question
# step 2 -> get the resultant tables.. with default nearest value as 2
# step 3 -> validate if the resultant tables are enough
# step 4 -> if not increase the nearest neighbour by 2 and get the data and validate it again

# step 5 -> get the column data with the default nearest neighbour value as 3
# step 6 -> validate the resulttant


table_graph_builder = StateGraph(TableState)
# graph_builder = StateGraph(State)

table_graph_builder.add_node("get_relevant_tables", get_relevant_tables)
table_graph_builder.add_node("validate_tables", validate_relevant_tables)
table_graph_builder.add_node("send_tables", send_tables)
table_graph_builder.add_node("not_possible_answer", not_possible_answer)
table_graph_builder.add_node("increase_relevance", increase_relevance)

table_graph_builder.add_edge(START, "get_relevant_tables")
table_graph_builder.add_edge("get_relevant_tables", "validate_tables")
table_graph_builder.add_conditional_edges("validate_tables", transition_by_validation, {"increase_relevence and reiletrate" : "increase_relevance", "answer_found" : "send_tables", "answer_can_not_found": "not_possible_answer"})
table_graph_builder.add_edge("increase_relevance", "get_relevant_tables")
table_graph_builder.add_edge("send_tables", END)
table_graph_builder.add_edge("not_possible_answer", END)

table_gen_agent = table_graph_builder.compile()


# 1 - get back and get more similar results
# 2 - output the result
# 3 - getting output is not possible

# graph_builder.add_node("write_query", write_query)
# graph_builder.add_node("validate_query", validate_query)
# graph_builder.add_node("execute_query", execute_query)
# graph_builder.add_node("generate_answer", generate_answer)

# graph_builder.add_edge(START, "write_query")
# graph_builder.add_edge("write_query", "validate_query")

# graph_builder.add_conditional_edges(
#     "validate_query",
#     transition_by_error,
#     {"valid_query":"execute_query",
#      "not_valid_query":"write_query"}
# )

# graph_builder.add_edge("execute_query", "generate_answer")
# graph_builder.add_edge("generate_answer", END)

# graph = graph_builder.compile()
# sql_agent

In [ ]:
from IPython.display import Image, display

# Save graph to file
graph_path = "_table_gen_agent_graph.png"
with open(graph_path, "wb") as f:
    f.write(table_gen_agent.get_graph().draw_mermaid_png())

# Display the saved image (clean output)
display(Image(filename=graph_path))

In [ ]:
for step in table_gen_agent.stream(
    {"question": "tell me 5 most likeable movies by our young users?", "relevant_tables": 1, "vectorstore": vector_store}, stream_mode="updates"
):
    print(step)

Table Generation Workflow is done and the output is amazing I am getting all the related tables.. as you can see in resultant_tables

Started Working on Related Column Generation Workflow

In [ ]:
class ColumnState(TypedDict):
    question : str # user query
    relevant_columns : int  # no of relevant columns
    column_documents : list # columns documents which are going to fetch
    validation_result : dict # validation result
    input_tables : list # list of columns to get the documents
    result : str # final output
    error_result : str # if the relevant tables can not found.
    vectorstore : FAISS

In [ ]:
def get_relevant_columns(state: ColumnState):
  vector_store = state["vectorstore"]
  res = vector_store.similarity_search(
    state["question"],
    k=state["relevant_columns"],
    filter={"source": "column_schema", "table_name": {"$in": state["input_tables"]}},
  )

  return {"column_documents": res}

In [ ]:
def validate_relevant_tables_and_columns(state : ColumnState):
  system_prompt = """
You are a helpful assistant that evaluates whether a given set of database tables contains enough schema information to generate a valid SQL query that answers a natural language question **in full**.

You will be provided with:
- A natural language question.
- One or more database tables. Each table includes:
  - Table name and description
  - Column names, data types, and column descriptions

Assume that:
- All columns are populated with valid and correctly typed data.
- You are not required to write the SQL query or answer the question — only to determine whether it is possible to generate a complete query using the schema provided.

Your task is to:
1. Carefully analyze the natural language question to identify **all distinct information requirements** (e.g., filters, metrics, grouping, sorting).
2. Review the table schemas and ensure that **every required data point** is represented by one or more columns in the schema.
3. Consider relationships between tables only if they are explicitly defined or logically inferable from naming or descriptions.

Only if **every part** of the question can be addressed with the schema provided, return:
{{"enough_information": true}}

If **any required element is missing** (e.g., age of users, movie ratings, like counts, timestamps, etc.), return:
{{"enough_information": false}}

Constraints:
- You must evaluate the entire question in detail — partial answerability is not sufficient.
- Do not assume access to external data, hidden metadata, or business logic beyond the schema.
- Do not write or return SQL queries.
- Your output must be only a single JSON object.

Input format:
Question: {natural_language_question}

Table(s):
{table_name_and_description}

Example output:
{{"enough_information": true}}
"""


  column_documents = state["column_documents"]

  table_column_documents = {}
  for documents in column_documents:
    if documents.metadata["table_name"] not in table_column_documents:
      table_column_documents[documents.metadata["table_name"]] = "Table Name: "+documents.metadata["table_name"]+"\n"
      table_column_documents[documents.metadata["table_name"]] += "Description: "+documents.metadata["table_description"]
      table_column_documents[documents.metadata["table_name"]] += "\n\n"
      table_column_documents[documents.metadata["table_name"]] += "Columns:\n"
    table_column_documents[documents.metadata["table_name"]] += documents.page_content

  table_column_information = ""
  for tables in table_column_documents:
    table_column_information += "\n"+table_column_documents[tables]

  print(table_column_information)

  prompt = ChatPromptTemplate.from_template(system_prompt)

  structured_llm = llm.with_structured_output(QueryJsonOutput)

  validation_chain = prompt | structured_llm

  result = validation_chain.invoke({"natural_language_question" : state["question"], "table_name_and_description": table_column_information})

  return {"validation_result" : result}

In [ ]:
def transition_by_column_validation(state: ColumnState):
  if "enough_information" not in state["validation_result"]:
    raise Exception("llm is not returning correct json")
  if state["validation_result"]["enough_information"] is True:
    return "answer_found"
  else:
    if state["relevant_columns"]<MAX_COLUMN_SCHEMA:
      return "increase_relevence and reiletrate"
    else:
      return "answer_can_not_found"

In [ ]:
def increase_column_relevance(state: ColumnState):
  result = state["relevant_columns"] + 2
  return {"relevant_columns" : result}

def send_tables_information_as_string(state : ColumnState):
  column_documents = state["column_documents"]

  table_column_documents = {}
  for documents in column_documents:
    if documents.metadata["table_name"] not in table_column_documents:
      table_column_documents[documents.metadata["table_name"]] = "Table Name: "+documents.metadata["table_name"]+"\n"
      table_column_documents[documents.metadata["table_name"]] += "Description: "+documents.metadata["table_description"]
      table_column_documents[documents.metadata["table_name"]] += "\n\n"
      table_column_documents[documents.metadata["table_name"]] += "Columns:\n"
    table_column_documents[documents.metadata["table_name"]] += documents.page_content

  table_column_information = ""
  for tables in table_column_documents:
    table_column_information += "\n"+table_column_documents[tables]

  return {"result": table_column_information}

In [ ]:
graph_builder = StateGraph(ColumnState)
graph_builder.add_node("get_relevant_columns", get_relevant_columns)
graph_builder.add_node("validate_relevant_tables_and_columns", validate_relevant_tables_and_columns)
graph_builder.add_node("increase_relevance", increase_column_relevance)
graph_builder.add_node("send_tables_information_as_string", send_tables_information_as_string)
graph_builder.add_node("not_possible_answer", not_possible_answer)

graph_builder.add_edge(START, "get_relevant_columns")
graph_builder.add_edge("get_relevant_columns", "validate_relevant_tables_and_columns")
graph_builder.add_conditional_edges("validate_relevant_tables_and_columns", transition_by_column_validation, {"increase_relevence and reiletrate" : "increase_relevance", "answer_found" : "send_tables_information_as_string", "answer_can_not_found": "not_possible_answer"})
graph_builder.add_edge("increase_relevance", "get_relevant_columns")
graph_builder.add_edge("send_tables_information_as_string", END)
graph_builder.add_edge("not_possible_answer", END)


column_gen_agent = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

# Save graph to file
graph_path = "_column_gen_agent_graph.png"
with open(graph_path, "wb") as f:
    f.write(column_gen_agent.get_graph().draw_mermaid_png())

# Display the saved image (clean output)
display(Image(filename=graph_path))

In [ ]:
for step in column_gen_agent.stream(
    {"question": "top 5 ranked movies?", "relevant_columns": 4, "input_tables" : ['ratings', 'movies'], "vectorstore": vector_store}, stream_mode="updates"
):
    print(step)

Can get the output in any fomat but chosen simple string as our next workflow will be needing the output in the simple plain text

In [ ]:
#plan of action: For getting answers from a query

In [ ]:
# user query ask ->

# Node-1 -> Query generation Agent -> take the llm , table description and prompt and generate a sqlite query
# Node-2 -> Validate the query -> run the query in sqlite and checks if it gives any error or not if yes return the error and move for refinement
# if no move to llm to generate the output
# Node-3 -> Refine the query -> take the error , table description and prompt and do the query generation again and move back to agent-2

# After getting the output
# Node-4 -> Geneate the final output and return it using a table.


Creation of an Sql Agent Begins -> query generator validator and executor

In [ ]:
from typing_extensions import Annotated
from typing_extensions import TypedDict

class Error(TypedDict):
    iserror : bool
    error : str

class SqlState(TypedDict):
    question: str
    query: str
    result: str
    answer: str
    top_k: int
    table_schema : str
    dialect : str
    error : Error

class QueryOutput(TypedDict):
    """Generated SQL query."""
    query: Annotated[str, ..., "Syntactically valid SQL query."]

def write_query(state: SqlState):
    """Generate SQL query to fetch information."""
    if 'error' in state and 'iserror' in state['error'] and state['error']['iserror']:

      system_error_prompt = """
          You are an expert SQL query assistant. Your task is to analyze a provided SQL query, along with a natural language question, error description, and table schema (with column information). You must identify issues in the SQL query and generate a corrected, syntactically valid SQL query that fulfills the user’s intent based on the given SQL dialect. Follow these strict instructions:

          What You Receive as Input:
          - A natural language question representing the user's intent.
          - A raw SQL query generated for that question.
          - An error message (if any) generated during execution of the query.
          - A schema, which includes table names and the list of valid columns for each table.
          - The required SQL dialect (e.g., PostgreSQL, MySQL, SQLite, etc.).
          - (Optional) A number of examples the user wants in the result.

          What You Must Do:
          1. Understand the Intent:
            - Parse the natural language question to determine the user’s actual intent and what data they are trying to retrieve.

          2. Validate the Original Query:
            - Compare the SQL query against the schema.
            - Identify misuse of columns, incorrect table joins, or syntax errors.
            - Use the provided error message to help locate the issue (e.g., missing columns, ambiguous names, invalid syntax, unsupported features).

          3. Correct the Query:
            - Rewrite the SQL query to fix any issues while preserving the user’s intent.
            - Ensure the query is syntactically correct for the specified SQL dialect.

          4. Apply Output Constraints:
            - Only use columns and tables explicitly defined in the schema. Do not use any undefined columns.
            - Be aware of which columns belong to which tables to avoid ambiguous or invalid references.
            - If the user specifies a number of examples (top K results), apply an appropriate LIMIT K clause.
            - Sort the results using meaningful columns (e.g., created_at, score, or other relevance indicators) to provide the most informative and relevant answers.
            - Ensure the query is well-structured and readable.

          Rules to Follow:
          - Always use only the columns listed in the schema.
          - Be aware of which columns belong to which tables to avoid invalid references.
          - Choose sorting columns that provide semantic value or align with the user's question.
          - Follow the syntax and functions specific to the provided SQL dialect.
          - If LIMIT is not specified and no number of examples is given, default to LIMIT 10.
          - Use fully qualified column names (table.column) where needed for clarity and correctness.

          Output Format:
          Your response should include only the final corrected SQL query, formatted properly. Do not include explanations or extra text unless specifically asked for.
      """

      user_prompt = """Input Information:
          - Natural language question: {question}
          - Original SQL query: {original_query}
          - Error message (if any): {error_message}
          - SQL dialect: {dialect}
          - Number of examples requested (if any): {k_examples}
          - Schema (tables and columns): {schema}
      """

      error_prompt_template = ChatPromptTemplate(
          [("system", system_error_prompt), ("user", user_prompt)]
      )

      prompt = error_prompt_template.invoke(
          {
              "error_message" : state['error']['error'],
              "original_query" : state['query'],
              "dialect": state["dialect"],
              "k_examples": str(state["top_k"]),
              "schema": state["table_schema"],
              "question": state["question"],
          }
      )

    else:

      system_message = """
        Given an input question, create a syntactically correct {dialect} query to
        run to help find the answer. Unless the user specifies in his question a
        specific number of examples they wish to obtain, ensure the query limits the output to
        no more than {top_k} entries. You can order the results by a relevant column to
        return the most interesting examples in the database.

        Never query for all the columns from a specific table, only ask for a the
        few relevant columns given the question.

        Pay attention to use only the column names that you can see in the schema
        description. Be careful to not query for columns that do not exist. Also,
        pay attention to which column is in which table.

        Only use the following tables:
        {table_info}
      """

      user_prompt = "Question: {input}"

      query_prompt_template = ChatPromptTemplate(
          [("system", system_message), ("user", user_prompt)]
      )

      prompt = query_prompt_template.invoke(
          {
              "dialect": state["dialect"],
              "top_k": str(state["top_k"]),
              "table_info": state["table_schema"],
              "input": state["question"],
          }
      )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

def execute_query(state: SqlState):
    """Execute SQL query."""
    conn = sqlite3.connect('real_database.db')
    cursor = conn.cursor()
    cursor.execute(state["query"])
    rows = cursor.fetchall()
    conn.commit()
    conn.close()
    return {"result": rows}

    # execute_query_tool = QuerySQLDatabaseTool(db=db)
    # return {"result": execute_query_tool.invoke(state["query"])}

In [ ]:
def validate_query(state: SqlState):
  """Validate Sql Query"""
  conn = sqlite3.connect('dummy_database.db')
  cursor = conn.cursor()
  try:
    cursor.execute(state["query"])
    rows = cursor.fetchall()
    return {"error" : {"iserror": False, "error": ''}}
  except Exception as e:
    return {"error" : {"iserror": True, "error": str(e)}}
  finally:
    cursor.close()
    conn.close()

In [ ]:
execute_query({"query": "SELECT count(DISTINCT movie_id) FROM ratings LIMIT 10"})

In [ ]:
def generate_answer(state: SqlState):
    """Answer question using retrieved information as context."""
    prompt = (
        "You are a data interpretation assistant.\n\n"
        "Answer the user's question clearly and directly using only the provided data.\n\n"
        "If you cannot find the answer from the provided data, simply state:\n"
        "\"I cannot provide the answer using the provided data.\"\n"
        "Do not give any explanation or reasoning.\n\n"
        "If you can find the answer, do not mention SQL queries or SQL results in your response. Instead, provide:\n"
        "- A clear, final answer that directly addresses the user's question.\n"
        "- A brief explanation of how the answer was derived, including what in the data supports it (but not the query used).\n\n"
        "Keep your response focused, factual, and based solely on the provided information."
        f"Question: {state['question']}\n"
        f"SQL Query: {state['query']}\n"
        f"SQL Result: {state['result']}"
    )
    print("query : ", state['query'])
    print("result : ", state['result'])
    response = llm.invoke(prompt)
    return {"answer": response.content}

In [ ]:
def transition_by_error(state: SqlState):
  if state["error"]["iserror"] is True:
    return "not_valid_query"
  else:
    return "valid_query"

In [ ]:
from langgraph.graph import START,END, StateGraph


sql_graph_builder = StateGraph(SqlState)

sql_graph_builder.add_node("write_query", write_query)
sql_graph_builder.add_node("validate_query", validate_query)
sql_graph_builder.add_node("execute_query", execute_query)
sql_graph_builder.add_node("generate_answer", generate_answer)

sql_graph_builder.add_edge(START, "write_query")
sql_graph_builder.add_edge("write_query", "validate_query")

sql_graph_builder.add_conditional_edges(
    "validate_query",
    transition_by_error,
    {"valid_query":"execute_query",
     "not_valid_query":"write_query"}
)

sql_graph_builder.add_edge("execute_query", "generate_answer")
sql_graph_builder.add_edge("generate_answer", END)

sql_agent = sql_graph_builder.compile()
# sql_agent

In [ ]:
from IPython.display import Image, display

display(Image(sql_agent.get_graph().draw_mermaid_png()))

In [ ]:
def get_dialect():
  conn = sqlite3.connect(":memory:")
  # Create a cursor to interact with the database
  cursor = conn.cursor()
  # Get the SQLite version
  cursor.execute("select sqlite_version();")
  version = cursor.fetchone()[0]
  return 'sqlite : '+version

In [ ]:
sc = '\nTable Name: movies\nDescription: This table includes metadata related to the movies.\n\nColumns:\n- movie_id (INTEGER): Unique movie identifier- movie_title (TEXT): Title of the movie (with release year)\nTable Name: ratings\nDescription: This table captures the ratings provided by users for specific movies.\n\nColumns:\n- user_rating (INTEGER): Rating from 1 to 5 (whole stars only)- movie_id (INTEGER): Foreign key referring to movies table'
for step in sql_agent.stream(
    {"question": "top 5 ranked movies?","dialect":get_dialect(),"top_k":5,"table_schema":sc}, stream_mode="updates"
):
    print(step)

In [ ]:
# for step in sql_agent.stream(
#     {"question": "Backdraft released in?","dialect":get_dialect(),"top_k":5,"table_schema":sc}, stream_mode="updates"
# ):
#     print(step)

In [ ]:
# for step in sql_agent.stream(
#     {"question": "how many users are unsatisfied with the movies?","dialect":get_dialect(),"top_k":5,"table_schema":sc}, stream_mode="updates"
# ):
#     print(step)

In [ ]:
# for step in sql_agent.stream(
#     {"question": "how many users are satisfied with the movies?","dialect":get_dialect(),"top_k":5,"table_schema":sc}, stream_mode="updates"
# ):
#     print(step)

In [ ]:
# for step in sql_agent.stream(
#     {"question": "tell me 5 most likeable movies by our old users?","dialect":get_dialect(),"top_k":5,"table_schema":sc}, stream_mode="updates"
# ):
#     print(step)

In [ ]:
# for step in sql_agent.stream(
#     {"question": "tell me 5 most likeable movies by our young users?","dialect":get_dialect(),"top_k":5,"table_schema":sc}, stream_mode="updates"
# ):
#     print(step)

In [ ]:
# for step in sql_agent.stream(
#     {"question": "tell me 5 most likeable movies by our mid age users?","dialect":get_dialect(),"top_k":5,"table_schema":sc}, stream_mode="updates"
# ):
#     print(step)

In [ ]:
# for step in sql_agent.stream(
#     {"question": "top 5 ranked movies?","dialect":get_dialect(),"top_k":5,"table_schema":sc}, stream_mode="updates"
# ):
#     print(step)

In [ ]:
# res = sql_agent.invoke(
#     {"question": "top 5 ranked movies?","dialect":get_dialect(),"top_k":5,"table_schema":sc}
# )


Combined them together to create a multi-agentic workflow

In [ ]:
class State(TypedDict):
    question : str # user query
    fetched_tables : list
    table_information : str
    init_relevant_tables: int
    init_relevant_columns : int
    vectorstore: FAISS
    relevant_columns : list
    dialect: str
    top_k : int
    sql_agent_output : str
    final_output : str
    error : str
    iserror : bool

In [ ]:
def invoke_get_table_agaent(state : State):
  print("passed_values", {"question": state["question"], "relevant_tables": state["init_relevant_tables"], "vectorstore": state["vectorstore"]})
  res = table_gen_agent.invoke({"question": state["question"], "relevant_tables": state["init_relevant_tables"], "vectorstore": state["vectorstore"]})
  if 'error_result' in res:
    return {'error' : res['error_result'], 'iserror' : True}
  final_result = res['resultant_tables']
  return {'fetched_tables' : final_result}

In [ ]:
def invoke_get_column_agent(state: State):
  print({"question": state["question"], "relevant_columns": state["init_relevant_columns"],"input_tables" : state["fetched_tables"], "vectorstore": state["vectorstore"]})
  res = column_gen_agent.invoke({"question": state["question"], "relevant_columns": state["init_relevant_columns"],"input_tables" : state["fetched_tables"], "vectorstore": state["vectorstore"]})
  if 'error_result' in res:
    return {'error' : res['error_result'], 'iserror' : True}
  final_result = res['result']
  return {'table_information' : final_result}

In [ ]:
def get_final_output(state: State):
  res = sql_agent.invoke({"question": state["question"],"dialect": state["dialect"], "top_k":state["top_k"], "table_schema": state["table_information"]})
  final_result = res['answer']
  return {'sql_agent_output': final_result}

In [ ]:
def generate_final_answer(state: State):
  if "iserror" in state and state["iserror"]:
    return {'final_output' : state['error']}
  else:
    return {'final_output' : state['sql_agent_output']}

In [ ]:
def transition_state(state: State):
  if "iserror" in state and state["iserror"]:
    return "generate_output"
  else:
    return "next_step"

In [ ]:
combined_agent_builder = StateGraph(State)

combined_agent_builder.add_node("table_agent", invoke_get_table_agaent)
combined_agent_builder.add_node("column_agent", invoke_get_column_agent)
combined_agent_builder.add_node("sql_agent", get_final_output)
combined_agent_builder.add_node("generate_final_answer", generate_final_answer)

combined_agent_builder.add_edge(START, "table_agent")
combined_agent_builder.add_conditional_edges(
    "table_agent",
    transition_state,
    {"generate_output":"generate_final_answer",
     "next_step":"column_agent"}
)
combined_agent_builder.add_conditional_edges(
    "column_agent",
    transition_state,
    {"generate_output":"generate_final_answer",
     "next_step":"sql_agent"}
)
combined_agent_builder.add_edge("sql_agent", "generate_final_answer")
combined_agent_builder.add_edge("generate_final_answer", END)

combined_agent = combined_agent_builder.compile()

In [ ]:
from IPython.display import Image, display

# Save graph to file
graph_path = "_table_gen_agent_graph.png"
with open(graph_path, "wb") as f:
    f.write(table_gen_agent.get_graph().draw_mermaid_png())

# Display the saved image (clean output)
display(Image(filename=graph_path))

In [ ]:
from IPython.display import Image, display

# Save graph to file
graph_path = "_column_gen_agent_graph.png"
with open(graph_path, "wb") as f:
    f.write(column_gen_agent.get_graph().draw_mermaid_png())

# Display the saved image (clean output)
display(Image(filename=graph_path))

In [ ]:
from IPython.display import Image, display

# Save graph to file
graph_path = "_sql_agent_graph.png"
with open(graph_path, "wb") as f:
    f.write(sql_agent.get_graph().draw_mermaid_png())

# Display the saved image (clean output)
display(Image(filename=graph_path))

In [ ]:
from IPython.display import Image, display

# Save graph to file
graph_path = "_combined_agent_graph.png"
with open(graph_path, "wb") as f:
    f.write(combined_agent.get_graph().draw_mermaid_png())

# Display the saved image (clean output)
display(Image(filename=graph_path))

In [ ]:
# for step in table_gen_agent.stream(
#     {'question': 'top 5 ranked movies?', 'relevant_tables': 1, 'vectorstore': vector_store}, stream_mode="updates"
# ):
#     print(step)

In [ ]:
# for step in column_gen_agent.stream({"question": "top 5 ranked movies??", "relevant_columns": 1, "input_tables" : ['users', 'ratings', 'movies'], "vectorstore": vector_store}, stream_mode="updates"):
# column_gen_agent.invoke({"question": "top 5 ranked movies?", "relevant_columns": 2, "input_tables" : ['users', 'ratings', 'movies'], "vectorstore": vector_store})
    # print(step)

In [ ]:
# column_gen_agent.invoke({"question": "top 5 ranked movies?", "relevant_columns": 3, "input_tables" : ['users', 'ratings', 'movies'], "vectorstore": vector_store})


In [ ]:
combined_agent.invoke({"question": "Backdraft released in?", "dialect":get_dialect(), "top_k":5, "vectorstore":vector_store, "init_relevant_tables": 2, "init_relevant_columns": 4})